In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor
!pip install scipy
import scipy

  Using cached scipy-1.5.4-cp36-cp36m-manylinux1_x86_64.whl (25.9 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# Transfer Learning from MNIST

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
train_data = datasets.MNIST(root = 'data', train = True, transform = ToTensor(), download = True)
train_loader = torch.utils.data.DataLoader(train_data,batch_size=100,shuffle=True,num_workers=4)
    
test_data = datasets.MNIST(root = 'data', train = False,transform = ToTensor())
test_loader = torch.utils.data.DataLoader(test_data,batch_size=100,shuffle=True,num_workers=4)

5.0%

100.1%

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


3.5%5%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


...

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


180.4%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

In [6]:
cnn = CNN()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)

total_step = len(train_loader)
num_epochs = 10
        
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
            

        output = cnn(images)[0]               
        loss = loss_func(output, labels)
            
        # clear gradients for this training step   
        optimizer.zero_grad()           

        # backpropagation, compute gradients 
        loss.backward()    
        # apply gradients             
        optimizer.step()                

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 0.2001
Epoch [1/10], Step [200/600], Loss: 0.0517
Epoch [1/10], Step [300/600], Loss: 0.0540
Epoch [1/10], Step [400/600], Loss: 0.0395
Epoch [1/10], Step [500/600], Loss: 0.0515
Epoch [1/10], Step [600/600], Loss: 0.0348
Epoch [2/10], Step [100/600], Loss: 0.0959
Epoch [2/10], Step [200/600], Loss: 0.0951
Epoch [2/10], Step [300/600], Loss: 0.0879
Epoch [2/10], Step [400/600], Loss: 0.0399
Epoch [2/10], Step [500/600], Loss: 0.1312
Epoch [2/10], Step [600/600], Loss: 0.0584
Epoch [3/10], Step [100/600], Loss: 0.0249
Epoch [3/10], Step [200/600], Loss: 0.0156
Epoch [3/10], Step [300/600], Loss: 0.0096
Epoch [3/10], Step [400/600], Loss: 0.0132
Epoch [3/10], Step [500/600], Loss: 0.0665
Epoch [3/10], Step [600/600], Loss: 0.0283
Epoch [4/10], Step [100/600], Loss: 0.0429
Epoch [4/10], Step [200/600], Loss: 0.0625
Epoch [4/10], Step [300/600], Loss: 0.0104
Epoch [4/10], Step [400/600], Loss: 0.0457
Epoch [4/10], Step [500/600], Loss: 0.0108
Epoch [4/10

In [8]:
with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)

Test Accuracy of the model on the 10000 test images: 0.98


In [24]:
svhn_transform = transforms.Compose([
#     transforms.Resize(256),
    transforms.CenterCrop(28),
    transforms.Grayscale(),
    transforms.ToTensor(),
])

svhn_data = datasets.SVHN(root = 'data', split = 'test',transform = svhn_transform, download = True)
svhn_loader = torch.utils.data.DataLoader(svhn_data,batch_size=100,shuffle=True,num_workers=0)

Using downloaded and verified file: data/test_32x32.mat


In [32]:
with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in svhn_loader:
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)

Test Accuracy of the model on the 10000 test images: 0.22
